In [1]:
import os
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import numpy as np
from collections import Counter

In [2]:
# Change the current working directory to the project root folder
# This ensures the code works with relative paths and avoids FileNotFoundError
os.chdir("../")
%pwd

'd:\\Projects\\Apple-Disease-Classification-Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    built_model_path: Path
    trained_model_path: Path    
    training_data: Path
    all_params: dict

In [8]:
from cnnClassifier.constants import *
from keras.optimizers import RMSprop 
from cnnClassifier.utils.common import read_yaml, create_directories
from src.cnnClassifier.logging import logger

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        model_preparation = self.config.model_preparation
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "dataset")
        create_directories([ Path(training.root_dir) ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            built_model_path=Path(model_preparation.built_model_path),
            trained_model_path=Path(training.trained_model_path),
            training_data=Path(training_data),
            all_params=self.params, 
        )

        return training_config

In [6]:
class TrainModel:
    def __init__(self, config: TrainingConfig):
        # Initialize the training configuration
        self.config = config

    def data_generator(self):
        """
        Load and preprocess the training and validation data.

        Returns:
            train_data, val_data: The training and validation data generators.
        """
        # Implement data loading and preprocessing:
        train_datagen = ImageDataGenerator(
            rescale=1./255,               # Rescale pixel values to [0, 1]
            rotation_range=10,            # Rotate images randomly by up to 10 degrees
            width_shift_range=0.1,        # Shift images horizontally by up to 10% of the width
            zoom_range=0.1,               # Zoom in or out by up to 10%
            horizontal_flip=True,         # Flip images horizontally (left to right)
            )
        
        self.train_generator = train_datagen.flow_from_directory(
            f'{self.config.training_data}/train',
            target_size=self.config.all_params.IMAGE_SIZE[:-1],
            batch_size=self.config.all_params.BATCH_SIZE,
            class_mode='sparse',
            shuffle=True,
            )

        validation_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=10,
            horizontal_flip=True)
            
        self.validation_generator = validation_datagen.flow_from_directory(
            f'{self.config.training_data}/val',
            target_size=self.config.all_params.IMAGE_SIZE[:-1],
            batch_size=self.config.all_params.BATCH_SIZE,
            class_mode='sparse',
            shuffle=False,
            )
    
    
    def calculate_class_weights(self):
        """
        Calculate class weights to handle class imbalance.
        """
        y_train = self.train_generator.classes
        class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
        self.class_weights = dict(enumerate(class_weights))


    def get_built_model(self):
        # Load the built model from the specified path
        self.model = tf.keras.models.load_model(
            self.config.built_model_path
        )

    def train_model(self):
        """
        Train the model using the loaded data and specified parameters.
        Save the trained model to the specified path.
        """
        # Compile the model
        optimizer = RMSprop(
            learning_rate=self.config.all_params.LEARNING_RATE,
            rho=self.config.all_params.RHO,
            epsilon=self.config.all_params.EPSILON
        )
        self.model.compile(
            optimizer=optimizer, 
            loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
            metrics=["accuracy"]
        )
    
        # Define the learning rate annealer
        reduce_lr = ReduceLROnPlateau(
            monitor=self.config.all_params.REDUCE_LR_MONITOR,
            patience=self.config.all_params.REDUCE_LR_PATIENCE,
            verbose=self.config.all_params.REDUCE_LR_VERPOSE,
            factor=self.config.all_params.REDUCE_LR_FACTOR,
            min_lr=self.config.all_params.REDUCE_LR_MIN_LR
        )

        # Define Early Stopping
        early_stop = EarlyStopping(
            monitor=self.config.all_params.EARLY_STOPPING_MONITOR,
            patience=self.config.all_params.EARLY_STOPPING_PATIENCE,
            mode=self.config.all_params.EARLY_STOPPING_MODE,
            restore_best_weights=self.config.all_params.RESTORE_BEST_WEIGHTS
        )

        # Calculate steps per epoch and validation steps
        steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        validation_steps = self.validation_generator.samples // self.validation_generator.batch_size

        # Train the model
        history = self.model.fit(
            self.train_generator,
            epochs=self.config.all_params.EPOCHS,
            validation_data=self.validation_generator,
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps,
            callbacks=[reduce_lr, early_stop],
            class_weight=self.class_weights
        )

        '''
        I tried using modlel.save(path), but I got "Deserializing Error", so I have to save the model with another way: 
        If aligning versions is not feasible, you can modify the model saving and loading code to handle the deserialization issue.        
        When saving the model, use the save_weights method and save the model architecture separately. 
        This ensures compatibility across different versions of TensorFlow/Keras.
        '''

        # Save the trained model architecture to JSON
        model_json = self.model.to_json()
        
        with open(self.config.trained_model_path.with_suffix(".json"), "w") as json_file:
            json_file.write(model_json)

        # Save model weights
        self.model.save_weights(self.config.trained_model_path.with_suffix(".h5"))

In [7]:
try:
    # Initialize the configuration manager to load config and params files
    config = ConfigurationManager()
    
    # Get the training configuration from the configuration manager
    training_config = config.get_training_config()
    
    # Initialize the Training class with the training configuration
    training = TrainModel(config=training_config)
    
    # Load the built model
    training.get_built_model()
    
    # Prepare the training and validation data generators
    training.data_generator()

    # Calculate class weights to handle class imbalance.
    training.calculate_class_weights()
    
    # Train the model
    training.train_model()

except Exception as e:
    # Raise any exceptions that occur during the process
    raise e

[2024-08-04 17:26:41,346: INFO: common: YAML file: configYaml\config.yaml loaded successfully]
[2024-08-04 17:26:41,350: INFO: common: YAML file: configYaml\params.yaml loaded successfully]
[2024-08-04 17:26:41,351: INFO: common: Created directory at: artifacts]
[2024-08-04 17:26:41,352: INFO: common: Created directory at: artifacts\training]
Found 3067 images belonging to 4 classes.
Found 437 images belonging to 4 classes.
95/95 [==============================] - 1215s 13s/step - loss: 1.7414 - accuracy: 0.1957 - val_loss: 2.5160 - val_accuracy: 1.0000 - lr: 0.0010
